In [ ]:
from selenium import webdriver
import time
from imgurpython import ImgurClient #imgur
import PIL
import configparser #匯入config套件
from selenium.webdriver.support.ui import Select
import MySQLdb
#config檔案匯入
config = configparser.ConfigParser()
config.read("./config.ini")
client_id=(config['imgur']['client_id'])
client_secret=(config['imgur']['client_secret'])
album_id=(config['imgur']['album_id'])
access_token=(config['imgur']['access_token'])
refresh_token=(config['imgur']['refresh_token'])

# 開啟Chrome瀏覽器
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--disable-gpu')
prefs = {"profile.managed_default_content_settings.images":2}
chrome_options.add_experimental_option("prefs",prefs)
browser = webdriver.Chrome(chrome_options=chrome_options) #有介面啟動
#browser = webdriver.Chrome(chrome_options=chrome_options) #無介面啟動

In [ ]:
url="127.0.0.1"
Acc = "test1234@gmail.com"
Pwd = "123456789"

In [ ]:
browser.get(url)
time.sleep(5)

while True: #登入點擊
    try:
        if browser.find_element_by_xpath("//a[@class='btn btn-primary btn-lg wow fadeInUp animated animated']"):
            browser.find_element_by_xpath("//a[@class='btn btn-primary btn-lg wow fadeInUp animated animated']").click()
            break
    except:
        print(f"再次嘗試")
    time.sleep(0.5)

while True:# 自動填入帳號密碼
    try:
        if browser.find_element_by_id("email"):
            browser.find_element_by_id("email").send_keys(Acc)
            browser.find_element_by_id("password").send_keys(Pwd)
            print(f"自動填入帳號密碼成功")
            break
    except:
        print(f"再次嘗試填入帳號密碼")
    time.sleep(0.5)

while True:# 按登入
    try:
        if browser.find_element_by_xpath("//button[@class='btn btn-primary']"):
            browser.find_element_by_xpath("//button[@class='btn btn-primary']").click()
            print(f"點擊登入成功")
            time.sleep(3)#登入後三秒轉跳
            break
    except:
        print(f"再次嘗試點擊登入")
    time.sleep(0.5)

time.sleep(1)


In [4]:
def rebackimage(jar_number):
    
    jar_id=jar_number
    
    while True:
        try:
            if Select(browser.find_element_by_name('jar_dashboard_id')):  # 例項化Select
                s1 = Select(browser.find_element_by_name('jar_dashboard_id'))  # 例項化Select
                s1.select_by_visible_text(str(jar_id))
                break
        except:
            print(f"再次嘗試選擇魚缸")
        time.sleep(0.5)
    while True:
        try:
            if browser.find_element_by_xpath("//canvas[@id='barChart']"):
                time.sleep(1)
                a = browser.find_element_by_xpath("//canvas[@id='barChart']")
                A=a.screenshot("code.png")
                print(f"擷取圖表成功")
                break
        except:
            print(f"嘗試擷取圖表")
        time.sleep(1)
    #imgur 上傳
    client = ImgurClient(client_id, client_secret, access_token, refresh_token)
    img_config = {
            'album': album_id,
            'name':'',
            'title':'',
            'description': '圖表'}
    while True:
        try:
            if A==a.screenshot("code.png"):
                print("Uploading image... ")
                image = client.upload_from_path('code.png', config=img_config, anon=False)
                print("Done")
                print(image['link'])
                imagelink = image['link']
                break
        except:
            print(f"fail")
        time.sleep(0.5)

    t = time.localtime()
    result = time.strftime("%Y%m%d%H%M", t)
    db = MySQLdb.connect(host=config['mysql']['host'],user=config['mysql']['user'], passwd=config['mysql']['password'], db=config['mysql']['user'])
    cursor = db.cursor()
    sql= "INSERT INTO line_bot_image_url (jarid,imageURL,timestamp) VALUES ('"+ str(jar_id) +"','" + imagelink + "','" + result+ "')"
    cursor.execute(sql)
    db.commit()
    cursor.close()
    db.close()
    
    return 

In [5]:
def main():
    while True:
        rebackimage(1)
        rebackimage(2)
        time.sleep(300)

In [ ]:
main()